# HW3 DATASCI W261: Machine Learning at Scale 

* **Name:**  Megan Jasek
* **Email:**  meganjasek@ischool.berkeley.edu
* **Class Name:**  W261-2
* **Week Number:**  3
* **Date:**  5/31/16

**HW3.0.**
How do you merge  two sorted  lists/arrays of records of the form [key, value]? Where is this  used in Hadoop MapReduce? [Hint within the shuffle]
What is  a combiner function in the context of Hadoop? 
Give an example where it can be used and justify why it should be used in the context of this problem.
What is the Hadoop shuffle?

**HW3.1** Consumer complaints dataset: Use Counters to do EDA (exploratory data analysis and to monitor progress.  
Now, let’s use Hadoop Counters to identify the number of complaints pertaining to debt collection, mortgage and other categories (all other categories get lumped into this one) in the consumer complaints dataset. Basically produce the distribution of the Product column in this dataset using counters (limited to 3 counters here).

Hadoop offers Job Tracker, an UI tool to determine the status and statistics of all jobs. Using the job tracker UI, developers can view the Counters that have been created. Screenshot your  job tracker UI as your job completes and include it here. Make sure that your user defined counters are visible. 

??add screen shot


In [29]:
# Read the Consumer_Complaints.csv input file into HDFS
!/usr/local/hadoop/bin/hdfs dfs -copyFromLocal Consumer_Complaints.csv /user/hadoop
!/usr/local/hadoop/bin/hdfs dfs -copyFromLocal Consumer_Complaints_small.csv /user/hadoop

16/05/27 00:04:22 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 18 items
-rw-r--r--   3 hadoop supergroup   50906486 2016-05-26 23:08 /user/hadoop/Consumer_Complaints.csv
-rw-r--r--   3 hadoop supergroup      16478 2016-05-27 00:04 /user/hadoop/Consumer_Complaints_small.csv
-rw-r--r--   3 hadoop supergroup     203981 2016-05-21 14:24 /user/hadoop/enronemail_1h.txt
drwxr-xr-x   - hadoop supergroup          0 2016-05-26 12:20 /user/hadoop/outputHW2-0-1
drwxr-xr-x   - hadoop supergroup          0 2016-05-21 14:06 /user/hadoop/outputHW2-1
drwxr-xr-x   - hadoop supergroup          0 2016-05-25 23:36 /user/hadoop/outputHW2-2
drwxr-xr-x   - hadoop supergroup          0 2016-05-26 12:57 /user/hadoop/outputHW2-2-1
drwxr-xr-x   - hadoop supergroup          0 2016-05-21 22:39 /user/hadoop/outputHW2-3
drwxr-xr-x   - hadoop supergroup          0 2016-05-25 11:35 /user/hadoop/outputHW2-3a
drwxr-xr-x   - hadoop 

In [6]:
# List out what is in the /user/hadoop directory in HDFS
!/usr/local/hadoop/bin/hdfs dfs -ls /user/hadoop

16/05/27 12:42:55 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 19 items
-rw-r--r--   3 hadoop supergroup   50906486 2016-05-26 23:08 /user/hadoop/Consumer_Complaints.csv
-rw-r--r--   3 hadoop supergroup      16478 2016-05-27 00:04 /user/hadoop/Consumer_Complaints_small.csv
-rw-r--r--   3 hadoop supergroup     203981 2016-05-21 14:24 /user/hadoop/enronemail_1h.txt
-rw-r--r--   3 hadoop supergroup         31 2016-05-27 12:42 /user/hadoop/hw3-2_strings.txt
drwxr-xr-x   - hadoop supergroup          0 2016-05-27 11:54 /user/hadoop/outputHW2-0-1
drwxr-xr-x   - hadoop supergroup          0 2016-05-27 11:56 /user/hadoop/outputHW2-1
drwxr-xr-x   - hadoop supergroup          0 2016-05-25 23:36 /user/hadoop/outputHW2-2
drwxr-xr-x   - hadoop supergroup          0 2016-05-26 12:57 /user/hadoop/outputHW2-2-1
drwxr-xr-x   - hadoop supergroup          0 2016-05-21 22:39 /user/hadoop/outputHW2-3
drwxr-xr-x   - ha

In [1]:
%%writefile mapper.py
#!/usr/bin/env python
## mapper.py
## Author: Megan Jasek
## Description: mapper code for HW3.1
## The function takes the Consumer Complaints file as input and reads the Product column and
## counts the different types of products.  The categories are:  'debt collection', 'mortgage' and 'others'

import sys

# Create a counter called Mapper Counters that will count the calls to function
sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")

# input comes from STDIN (standard input)
for line in sys.stdin:
    # Split the line on commas and store the product column value in the 'product' variable
    items = line.split(',')
    product = items[1].lower()
    # Skip the column header
    if product == "product":
        continue
    elif product == "debt collection":
        # Create a counter in the EDA Counters group called Debt that counts the products
        # in the debt collection group
        sys.stderr.write("reporter:counter:EDA Counters,Debt,1\n")
    elif product == "mortgage":
        # Create a counter in the EDA Counters group called Mortgage that counts the products
        # in the debt collection group
        sys.stderr.write("reporter:counter:EDA Counters,Mortgage,1\n")
    else:
        # Create a counter in the EDA Counters group called Others that counts the products
        # in all other groups
        sys.stderr.write("reporter:counter:EDA Counters,Others,1\n")
    # Print the product with a count of 1
    print('%s\t%d' % (product, 1))

Overwriting mapper.py


In [2]:
%%writefile reducer.py
#!/usr/bin/env python
## reducer.py
## Author: Megan Jasek
## Description: reducer code for HW3.1
## This function expects keys to come in sorted.  It sums the values of all keys that are the same,
## then it outputs each key with its total sum of all of its values.

import sys

# Create a counter called Reducer Counters that will count the calls to function
sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")

cur_key = None
cur_count = 0
for line in sys.stdin:
    # Split the line on tabs
    items = line.split('\t')
    # Store the 1st and 2nd items as key and value
    key = items[0]
    value = items[1]
    # If this key is the same as the previous key add the value to the count
    if key == cur_key:
        cur_count += int(value)
    # Otherwise, reset the current key and start a new count total
    else:
        if cur_key:
            print '%s\t%s' % (cur_key, cur_count)
        cur_key = key
        cur_count = int(value)

print '%s\t%s' % (cur_key, cur_count)

Overwriting reducer.py


In [3]:
# Remove output from previous runs of this command
!/usr/local/hadoop/bin/hdfs dfs -rm /user/hadoop/outputHW3-1/*
!/usr/local/hadoop/bin/hdfs dfs -rmdir /user/hadoop/outputHW3-1
# Run a Hadoop streaming job.  The input file 'Consumer_Complaints.csv' is passed as input.
!/usr/local/hadoop/bin/hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-files mapper.py,reducer.py -mapper mapper.py -reducer reducer.py \
-input /user/hadoop/Consumer_Complaints.csv -output /user/hadoop/outputHW3-1

16/05/27 14:43:58 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/27 14:43:58 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW3-1/_SUCCESS
16/05/27 14:43:58 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW3-1/part-00000
16/05/27 14:43:59 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/27 14:44:00 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/tmp/hadoop-unjar6987847830147326519/] [] /tmp/streamjob5627728082777250317.jar tmpDir=null
16/05/27 14:44:01 INFO client.RMProxy: Connecting to ResourceManager at master/50.97.205

In [4]:
# Print out what files were created from the MapReduce job
!/usr/local/hadoop/bin/hdfs dfs -ls /user/hadoop/outputHW3-1
# Print the output file
!/usr/local/hadoop/bin/hdfs dfs -cat /user/hadoop/outputHW3-1/part-00000

16/05/27 12:36:51 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 2 items
-rw-r--r--   3 hadoop supergroup          0 2016-05-27 12:36 /user/hadoop/outputHW3-1/_SUCCESS
-rw-r--r--   3 hadoop supergroup        184 2016-05-27 12:36 /user/hadoop/outputHW3-1/part-00000
16/05/27 12:36:53 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
bank account or service	38073
consumer loan	9387
credit card	41563
credit reporting	41214
debt collection	44372
money transfers	1540
mortgage	125752
payday loan	1579
student loan	9432


**HW 3.2** Analyze the performance of your Mappers, Combiners and Reducers using Counters.

**HW 3.2A** For this brief study the Input file will be one record (the next line only):  
foo foo quux labs foo bar quux

Perform a word count analysis of this single record dataset using a Mapper and Reducer based WordCount (i.e., no combiners are used here) using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing this word count job. The answer should be 1 and 4 respectively. Please explain.

ANSWER:  ??my counts are 2 and 2

Please use mulitple mappers and reducers for these jobs (at least 2 mappers and 2 reducers).

**HW 3.2B** Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper and Reducer based WordCount (i.e., no combiners used anywhere)  using user defined Counters to count up how many time the mapper and reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job.  
ANSWER:  Using 2 mappers and 2 reducers and NO combiner.  See hadoop output below marked 'OUTPUT FOR HW3.2B'.  Command used is as follows:  
!/usr/local/hadoop/bin/hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \  
-D mapreduce.job.maps=2 \  
-D mapred.reduce.tasks=2 \  
-files mapper.py,reducer.py -mapper mapper.py -reducer reducer.py \  
-input /user/hadoop/Consumer_Complaints.csv -output /user/hadoop/outputHW3-2b     

	Mapper Counters  
		Calls=2  
	Reducer Counters  
		Calls=2  

**HW 3.2C** Perform a word count analysis of the Issue column of the Consumer Complaints  Dataset using a Mapper, Reducer, and standalone combiner (i.e., not an in-memory combiner) based WordCount using user defined Counters to count up how many time the mapper, combiner, reducer are called. What is the value of your user defined Mapper Counter, and Reducer Counter after completing your word count job.  
ANSWER:  Using 2 mappers and 2 reducers and specifying a combiner.  See hadoop output below marked 'OUTPUT FOR HW3.2C'.  Command used is as follows:  
!/usr/local/hadoop/bin/hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \  
-D mapreduce.job.maps=2 \  
-D mapred.reduce.tasks=2 \  
-files mapper.py,reducer.py,combiner.py -mapper mapper.py -reducer reducer.py -combiner combiner.py \  
-input /user/hadoop/Consumer_Complaints.csv -output /user/hadoop/outputHW3-2c  

	Combiner Counters
		Calls=4
	Mapper Counters
		Calls=2
	Reducer Counters
		Calls=2

**HW 3.2D** Using a single reducer: What are the top 50 most frequent terms in your word count analysis? Present the top 50 terms and their frequency and their relative frequency.  If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items).  
ANSWER:  See hadoop output below marked 'OUTPUT FOR HW3.2D'.

In [15]:
# Read the required input file into HDFS
!/usr/local/hadoop/bin/hdfs dfs -rm /user/hadoop/hw3-2_strings.txt
!/usr/local/hadoop/bin/hdfs dfs -copyFromLocal hw3-2_strings.txt /user/hadoop

16/05/27 13:35:37 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/27 13:35:37 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/hw3-2_strings.txt
16/05/27 13:35:38 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [16]:
# List out what is in the /user/hadoop directory in HDFS
!/usr/local/hadoop/bin/hdfs dfs -cat /user/hadoop/hw3-2_strings.txt

16/05/27 13:35:42 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
foo foo quux labs foo bar quux


In [15]:
%%writefile mapper.py
#!/usr/bin/env python
## mapper.py
## Author: Megan Jasek
## Description: mapper code for HW3.2A

import sys

# Create a counter called Mapper Counters that will count the calls to function
sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")

# input comes from STDIN (standard input)
# Break up each line by space delimiter and print each word with a count of 1.
for line in sys.stdin:
    for word in line.split():
        print('%s\t%d' % (word, 1))

Overwriting mapper.py


In [5]:
%%writefile reducer.py
#!/usr/bin/env python
## reducer.py
## Author: Megan Jasek
## Description: reducer code for HW3.2A
## This function expects keys to come in sorted.  It sums the values of all keys that are the same,
## then it outputs each key with its total sum of all of its values.

import sys

# Create a counter called Reducer Counters that will count the calls to function
sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")

cur_key = None
cur_count = 0
for line in sys.stdin:
    # Split the line on tabs
    items = line.split('\t')
    # Store the 1st and 2nd items as key and value
    key = items[0]
    value = items[1]
    # If this key is the same as the previous key add the value to the count
    if key == cur_key:
        cur_count += int(value)
    # Otherwise, reset the current key and start a new count total
    else:
        if cur_key:
            print '%s\t%s' % (cur_key, cur_count)
        cur_key = key
        cur_count = int(value)

print '%s\t%s' % (cur_key, cur_count)

Overwriting reducer.py


In [14]:
# Remove output from previous runs of this command
!/usr/local/hadoop/bin/hdfs dfs -rm /user/hadoop/outputHW3-2a/*
!/usr/local/hadoop/bin/hdfs dfs -rmdir /user/hadoop/outputHW3-2a
# Run a Hadoop streaming job.  The input file 'Consumer_Complaints.csv' is passed as input.
!/usr/local/hadoop/bin/hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-files mapper.py,reducer.py -mapper mapper.py -reducer reducer.py \
-input /user/hadoop/hw3-2_strings.txt -output /user/hadoop/outputHW3-2a

16/05/27 13:34:40 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/27 13:34:40 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW3-2a/_SUCCESS
16/05/27 13:34:40 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW3-2a/part-00000
16/05/27 13:34:40 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW3-2a/part-00001
16/05/27 13:34:41 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/27 13:34:42 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJa

In [9]:
# Print out what files were created from the MapReduce job
!/usr/local/hadoop/bin/hdfs dfs -ls /user/hadoop/outputHW3-2a
# Print the output file
!/usr/local/hadoop/bin/hdfs dfs -cat /user/hadoop/outputHW3-2a/part-00000
!/usr/local/hadoop/bin/hdfs dfs -cat /user/hadoop/outputHW3-2a/part-00001
!/usr/local/hadoop/bin/hdfs dfs -cat /user/hadoop/outputHW3-2a/part-00002
!/usr/local/hadoop/bin/hdfs dfs -cat /user/hadoop/outputHW3-2a/part-00003

16/05/27 13:23:18 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 5 items
-rw-r--r--   3 hadoop supergroup          0 2016-05-27 13:22 /user/hadoop/outputHW3-2a/_SUCCESS
-rw-r--r--   3 hadoop supergroup          7 2016-05-27 13:22 /user/hadoop/outputHW3-2a/part-00000
-rw-r--r--   3 hadoop supergroup          6 2016-05-27 13:22 /user/hadoop/outputHW3-2a/part-00001
-rw-r--r--   3 hadoop supergroup          6 2016-05-27 13:22 /user/hadoop/outputHW3-2a/part-00002
-rw-r--r--   3 hadoop supergroup          7 2016-05-27 13:22 /user/hadoop/outputHW3-2a/part-00003
16/05/27 13:23:20 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
quux	2
16/05/27 13:23:21 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
foo	3
16/05/27 13:23:23 WARN util.NativeCodeLoa

In [20]:
%%writefile mapper.py
#!/usr/bin/env python
## mapper.py
## Author: Megan Jasek
## Description: mapper code for HW3.2B, HW3.2C

import sys

# Create a counter called Mapper Counters that will count the calls to function
sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")

# input comes from STDIN (standard input)
for line in sys.stdin:
    items = line.split(',')
    issue = items[3].lower()
    # If this is the column header, then do not count it.
    if issue == "issue":
        continue
    for word in issue.split():
        print('%s\t%d' % (word, 1))

Overwriting mapper.py


In [21]:
%%writefile combiner.py
#!/usr/bin/env python
## combiner.py
## Author: Megan Jasek
## Description: combiner code for HW3.2C

import sys

# Create a counter called Combiner Counters that will count the calls to function
sys.stderr.write("reporter:counter:Combiner Counters,Calls,1\n")

cur_key = None
cur_count = 0
for line in sys.stdin:
    items = line.split('\t')
    key = items[0]
    value = items[1]
    if key == cur_key:
        cur_count += int(value)
    else:
        if cur_key:
            print '%s\t%s' % (cur_key, cur_count)
        cur_key = key
        cur_count = int(value)

print '%s\t%s' % (cur_key, cur_count)

Overwriting combiner.py


In [22]:
%%writefile reducer.py
#!/usr/bin/env python
## reducer.py
## Author: Megan Jasek
## Description: reducer code for HW3.2B, HW3.2C
## This function expects keys to come in sorted.  It sums the values of all keys that are the same,
## then it outputs each key with its total sum of all of its values.

import sys

# Create a counter called Reducer Counters that will count the calls to function
sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")

cur_key = None
cur_count = 0
for line in sys.stdin:
    # Split the line on tabs
    items = line.split('\t')
    # Store the 1st and 2nd items as key and value
    key = items[0]
    value = items[1]
    # If this key is the same as the previous key add the value to the count
    if key == cur_key:
        cur_count += int(value)
    # Otherwise, reset the current key and start a new count total
    else:
        if cur_key:
            print '%s\t%s' % (cur_key, cur_count)
        cur_key = key
        cur_count = int(value)

print '%s\t%s' % (cur_key, cur_count)

Overwriting reducer.py


In [3]:
#################### OUTPUT FOR HW3.2B ########################
# Remove output from previous runs of this command
!/usr/local/hadoop/bin/hdfs dfs -rm /user/hadoop/outputHW3-2b/*
!/usr/local/hadoop/bin/hdfs dfs -rmdir /user/hadoop/outputHW3-2b
# Run a Hadoop streaming job.  The input file 'Consumer_Complaints.csv' is passed as input.
!/usr/local/hadoop/bin/hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D mapreduce.job.maps=2 \
-D mapred.reduce.tasks=2 \
-files mapper.py,reducer.py -mapper mapper.py -reducer reducer.py \
-input /user/hadoop/Consumer_Complaints.csv -output /user/hadoop/outputHW3-2b

16/05/27 17:58:05 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/27 17:58:06 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW3-2b/_SUCCESS
16/05/27 17:58:06 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW3-2b/part-00000
16/05/27 17:58:06 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW3-2b/part-00001
16/05/27 17:58:07 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/27 17:58:08 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJa

In [4]:
# Print out what files were created from the MapReduce job
!/usr/local/hadoop/bin/hdfs dfs -ls /user/hadoop/outputHW3-2b
# Print the output file
!/usr/local/hadoop/bin/hdfs dfs -cat /user/hadoop/outputHW3-2b/part-00000
!/usr/local/hadoop/bin/hdfs dfs -cat /user/hadoop/outputHW3-2b/part-00001

16/05/27 17:58:37 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 3 items
-rw-r--r--   3 hadoop supergroup          0 2016-05-27 17:58 /user/hadoop/outputHW3-2b/_SUCCESS
-rw-r--r--   3 hadoop supergroup        964 2016-05-27 17:58 /user/hadoop/outputHW3-2b/part-00000
-rw-r--r--   3 hadoop supergroup       1205 2016-05-27 17:58 /user/hadoop/outputHW3-2b/part-00001
16/05/27 17:58:39 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
"account	16205
/	12386
a	3503
account	4476
acct	163
an	2505
and	16448
applied	139
apr	3431
arbitration	168
available	274
bankruptcy	222
being	5663
billing	8158
by	5663
can't	1999
cash	240
caused	5663
changes	350
charges	131
checks	75
company's	4858
cont'd	11848
convenience	75
credit	50894
debt	19309
delay	243
delinquent	1061
deposits	10555
determination	1490
disclosure	5214
disputes	6938
expect	

In [23]:
#################### OUTPUT FOR HW3.2C ########################
# Remove output from previous runs of this command
!/usr/local/hadoop/bin/hdfs dfs -rm /user/hadoop/outputHW3-2c/*
!/usr/local/hadoop/bin/hdfs dfs -rmdir /user/hadoop/outputHW3-2c
# Run a Hadoop streaming job.  The input file 'Consumer_Complaints.csv' is passed as input.
!/usr/local/hadoop/bin/hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D mapreduce.job.maps=2 \
-D mapred.reduce.tasks=2 \
-files mapper.py,reducer.py,combiner.py -mapper mapper.py -reducer reducer.py -combiner combiner.py \
-input /user/hadoop/Consumer_Complaints.csv -output /user/hadoop/outputHW3-2c

16/05/28 13:00:16 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/28 13:00:16 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW3-2c/_SUCCESS
16/05/28 13:00:16 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW3-2c/part-00000
16/05/28 13:00:16 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW3-2c/part-00001
16/05/28 13:00:17 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/28 13:00:19 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJa

In [6]:
# Print out what files were created from the MapReduce job
!/usr/local/hadoop/bin/hdfs dfs -ls /user/hadoop/outputHW3-2c
# Print the output file
!/usr/local/hadoop/bin/hdfs dfs -cat /user/hadoop/outputHW3-2c/part-00000
!/usr/local/hadoop/bin/hdfs dfs -cat /user/hadoop/outputHW3-2c/part-00001

16/05/27 18:02:14 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Found 3 items
-rw-r--r--   3 hadoop supergroup          0 2016-05-27 18:01 /user/hadoop/outputHW3-2c/_SUCCESS
-rw-r--r--   3 hadoop supergroup        964 2016-05-27 18:01 /user/hadoop/outputHW3-2c/part-00000
-rw-r--r--   3 hadoop supergroup       1205 2016-05-27 18:01 /user/hadoop/outputHW3-2c/part-00001
16/05/27 18:02:16 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
"account	16205
/	12386
a	3503
account	4476
acct	163
an	2505
and	16448
applied	139
apr	3431
arbitration	168
available	274
bankruptcy	222
being	5663
billing	8158
by	5663
can't	1999
cash	240
caused	5663
changes	350
charges	131
checks	75
company's	4858
cont'd	11848
convenience	75
credit	50894
debt	19309
delay	243
delinquent	1061
deposits	10555
determination	1490
disclosure	5214
disputes	6938
expect	

In [9]:
# Copy the output of the Issues Column Word Count to separate files to use as input for sorting
!/usr/local/hadoop/bin/hdfs dfs -cp /user/hadoop/outputHW3-2c/part-00000 /user/hadoop/cc_wordcount_1
!/usr/local/hadoop/bin/hdfs dfs -cp /user/hadoop/outputHW3-2c/part-00001 /user/hadoop/cc_wordcount_2

16/05/27 18:05:03 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/27 18:05:04 WARN hdfs.DFSClient: DFSInputStream has been closed already


In [25]:
%%writefile mapper.py
#!/usr/bin/env python
## mapper.py
## Author: Megan Jasek
## Description: mapper code for HW3.2D
## This function passes the words and counts to the reducer.  It also sums up two other values:
## total word count (count_words) and total unique word count (count_unique).  And passes those
## values on to the reducer using special marker values 1000000000 and 2000000000.  This
## technique is called order inversion.

import sys

# Create a counter called Mapper Counters that will count the calls to function
sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")

count_words = 0
count_unique = 0

# input comes from STDIN (standard input)
# Sum up the total word count and total word count of unique words
# Print each word and its count for the reducer
for line in sys.stdin:
    # Split the line on tabs
    word, value = line.split('\t')
    count_words += int(value)
    count_unique += 1
    print('%s\t%d' % (word, int(value)))

# Print the special values 1000000000 and 2000000000 to mark the total count of words
# and the total unique count of words
print('%d\t1000000000' % (count_words))
print('%d\t2000000000' % (count_unique))

Overwriting mapper.py


In [26]:
%%writefile reducer.py
#!/usr/bin/env python
## reducer.py
## Author: Megan Jasek
## Description: reducer code for HW3.2D
## This function expects keys to come in sorted.  Special values from the mapper, values 1000000000
## and 2000000000 are used here to mark any total word count counts or unique word count counts.
## Because these values are so high, they will come in first from the mappers, so the total word
## count and total unique word count can be calculated first and then used for other calculations
## in the function.  The reducer can quickly calculate the relative frequency of a word
## by dividing by the total word count.  The top most frequent words are printed and the 10 least
## frequent words are printed.  The reducer uses the total unique word count to understand which
## words will be the 10 last in the sorted list.

import sys

# Create a counter called Reducer Counters that will count the calls to function
sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")

count_words = 0
count_unique = 0
i = 0
for line in sys.stdin:
    # Split the line on tabs
    items = line.split('\t')
    # Store the 1st and 2nd items as key and value
    key = items[0]
    value = int(items[1])
    # If the value equals the special number 1000000000, this indicates this is a total 
    # word count value and it should be added to the count_words total
    if value == 1000000000:
        count_words += float(key)
    # If the value equals the special number 2000000000, this indicates this is a unique 
    # word count value and it should be added to the count_unique total
    elif value == 2000000000:
        count_unique += float(key)
    else:
        # Print only the 50 most frequent and the 10 least frequent words
        if (i < 50) or (i >= (count_unique-10)):
            print('%s\t%s\t%f' % (key, value, value/count_words))
        i += 1

Overwriting reducer.py


In [16]:
# Remove output from previous runs of this command
!/usr/local/hadoop/bin/hdfs dfs -rm /user/hadoop/outputHW3-2d/*
!/usr/local/hadoop/bin/hdfs dfs -rmdir /user/hadoop/outputHW3-2d
# Run a Hadoop streaming job.  The input file 'Consumer_Complaints.csv' is passed as input.
!/usr/local/hadoop/bin/hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D stream.num.map.output.key.field=2 \
-D stream.map.output.field.separator="\t" \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D mapred.text.key.comparator.options="-k2,2nr -k1,1" \
-files mapper.py,reducer.py -mapper mapper.py -reducer reducer.py \
-input /user/hadoop/cc_wordcount_1,/user/hadoop/cc_wordcount_2 -output /user/hadoop/outputHW3-2d

16/05/28 12:50:24 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/28 12:50:25 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW3-2d/_SUCCESS
16/05/28 12:50:25 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.
Deleted /user/hadoop/outputHW3-2d/part-00000
16/05/28 12:50:26 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/28 12:50:27 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/tmp/hadoop-unjar3612921558270034290/] [] /tmp/streamjob7234082046083478946.jar tmpDir=null
16/05/28 12:50:28 INFO client.RMProxy: Connecting to ResourceManager at master/50.97.2

In [18]:
#################### OUTPUT FOR HW3.2D ########################
# Print the output file
!/usr/local/hadoop/bin/hdfs dfs -cat /user/hadoop/outputHW3-2d/part-00000

16/05/28 12:56:02 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
"loan	107254	0.109596
modification	70487	0.072026
credit	50894	0.052005
servicing	36767	0.037570
report	30546	0.031213
incorrect	29069	0.029704
information	29069	0.029704
on	29069	0.029704
or	22533	0.023025
debt	19309	0.019731
and	16448	0.016807
"account	16205	0.016559
opening	16205	0.016559
club	12545	0.012819
health	12545	0.012819
/	12386	0.012656
not	12353	0.012623
loan	12237	0.012504
attempts	11848	0.012107
collect	11848	0.012107
cont'd	11848	0.012107
owed	11848	0.012107
of	10885	0.011123
my	10731	0.010965
deposits	10555	0.010785
withdrawals	10555	0.010785
problems	9484	0.009691
"application	8625	0.008813
to	8401	0.008584
unable	8178	0.008357
billing	8158	0.008336
other	7886	0.008058
disputes	6938	0.007089
communication	6920	0.007071
tactics	6920	0.007071
reporting	6559	0.006702
lease	6337	0.006475
the	6248	0.006384
being	5663	0.005787


**HW3.2.1**  Using 2 reducers: What are the top 50 most frequent terms in your word count analysis? Present the top 50 terms and their frequency and their relative frequency. If there are ties please sort the tokens in alphanumeric/string order. Present bottom 10 tokens (least frequent items). Please use a combiner.

In [14]:
%%writefile mapper.py
#!/usr/bin/env python
## mapper.py
## Author: Megan Jasek
## Description: mapper code for HW3.2.1D
## This function passes the words and counts to the reducer.  It also sums up the
## total word count (count_words).  And passes it on to the reducer using special marker
## value 1000000000.  This technique is called order inversion.

import sys

# Create a counter called Mapper Counters that will count the calls to function
sys.stderr.write("reporter:counter:Mapper Counters,Calls,1\n")

count_words = 0

# input comes from STDIN (standard input)
# Sum up the total word count and total word count of unique words
# Print each word and its count for the reducer
for line in sys.stdin:
    # Split the line on tabs
    word, value = line.split('\t')
    if int(value) < 3000:
        label = "a"
    else:
        label = "b"
    count_words += int(value)
    print('%s\t%s\t%d' % (label, word, int(value)))

# Print the special value 1000000000 to mark the total count of words
print('a\t%d\t1000000000' % (count_words))
print('b\t%d\t1000000000' % (count_words))

Overwriting mapper.py


In [16]:
%%writefile reducer.py
#!/usr/bin/env python
## reducer.py
## Author: Megan Jasek
## Description: reducer code for HW3.2.1D
## This function expects keys to come in sorted.  Special values from the mapper, value 1000000000
## is used here to mark any total word count counts.
## Because this values is so high, it will come in first from the mappers, so the total word
## count and total unique word count can be calculated first and then used for other calculations
## in the function.  The reducer can quickly calculate the relative frequency of a word
## by dividing by the total word count.

import sys

# Create a counter called Reducer Counters that will count the calls to function
sys.stderr.write("reporter:counter:Reducer Counters,Calls,1\n")

count_words = 0
for line in sys.stdin:
    # Split the line on tabs
    items = line.split('\t')
    # Store the 1st and 2nd and 3rd items as label and key and value
    label = items[0]
    key = items[1]
    value = int(items[2])
    # If the value equals the special number 1000000000, this indicates this is a total 
    # word count value and it should be added to the count_words total
    if value == 1000000000:
        count_words += float(key)
    else:
        print('%s\t%s\t%f' % (key, value, value/count_words))

Overwriting reducer.py


In [21]:
# -combiner reducer.py
#-combiner Ireducer.py
#-D stream.num.map.output.key.field=2 \
#-D stream.map.output.field.separator="\t" \
#-D mapreduce.partition.keypartitioner.options=-k2,2 \
#-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
#-D mapred.text.key.comparator.options="-k2,2nr -k1,1" \


# Remove output from previous runs of this command
!/usr/local/hadoop/bin/hdfs dfs -rm /user/hadoop/outputHW3-2-1/*
!/usr/local/hadoop/bin/hdfs dfs -rmdir /user/hadoop/outputHW3-2-1
# Run a Hadoop streaming job.  The input file 'Consumer_Complaints.csv' is passed as input.
!/usr/local/hadoop/bin/hadoop jar /usr/local/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.1.jar \
-D stream.num.map.output.key.field=3 \
-D stream.map.output.field.separator="\t" \
-D mapreduce.partition.keypartitioner.options=-k1,1 \
-D mapred.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
-D mapred.text.key.comparator.options="-k3,3nr -k2,2" \
-D mapred.reduce.tasks=2 \
-files mapper.py,reducer.py -mapper mapper.py -reducer reducer.py \
-input /user/hadoop/cc_wordcount_1,/user/hadoop/cc_wordcount_2 -output /user/hadoop/outputHW3-2-1 \
-partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner

16/05/28 16:27:51 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
rm: `/user/hadoop/outputHW3-2-1/*': No such file or directory
16/05/28 16:27:53 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
16/05/28 16:27:54 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
packageJobJar: [/tmp/hadoop-unjar7883164916772480020/] [] /tmp/streamjob8177213495908193641.jar tmpDir=null
16/05/28 16:27:54 INFO client.RMProxy: Connecting to ResourceManager at master/50.97.205.254:8032
16/05/28 16:27:54 INFO client.RMProxy: Connecting to ResourceManager at master/50.97.205.254:8032
16/05/28 16:27:55 INFO mapred.FileInputFormat: Total input paths to process : 2
16/05/28 16:27:55 INFO mapreduce.JobSubmitter: number of splits:3
16/05/28 16:27:55 INFO Configuration.depreca

In [22]:
#################### OUTPUT FOR HW3.2.1D ########################
# Print the output file
!/usr/local/hadoop/bin/hdfs dfs -cat /user/hadoop/outputHW3-2-1/part-00000
!/usr/local/hadoop/bin/hdfs dfs -cat /user/hadoop/outputHW3-2-1/part-00001

16/05/28 16:29:53 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
"loan	107254	0.109596
modification	70487	0.072026
credit	50894	0.052005
servicing	36767	0.037570
report	30546	0.031213
incorrect	29069	0.029704
information	29069	0.029704
on	29069	0.029704
or	22533	0.023025
debt	19309	0.019731
and	16448	0.016807
"account	16205	0.016559
opening	16205	0.016559
club	12545	0.012819
health	12545	0.012819
/	12386	0.012656
not	12353	0.012623
loan	12237	0.012504
attempts	11848	0.012107
collect	11848	0.012107
cont'd	11848	0.012107
owed	11848	0.012107
of	10885	0.011123
my	10731	0.010965
deposits	10555	0.010785
withdrawals	10555	0.010785
problems	9484	0.009691
"application	8625	0.008813
to	8401	0.008584
unable	8178	0.008357
billing	8158	0.008336
other	7886	0.008058
disputes	6938	0.007089
communication	6920	0.007071
tactics	6920	0.007071
reporting	6559	0.006702
lease	6337	0.006475
the	6248	0.006384
being	5663	0.005787


In [59]:
### Testing cell
filename = "Consumer_Complaints_small.csv"
with open(filename, 'r') as f:
    for line in f.readlines():
        items = line.split(',')
        product = items[1]
        if product == "Debt collection":
            pass
        elif product == "Mortgage":
            pass
        else:
            pass
        print '%s\t%d' % (product, 1)

Product	1
Debt collection	1
Debt collection	1
Bank account or service	1
Debt collection	1
Credit reporting	1
Debt collection	1
Consumer loan	1
Debt collection	1
Consumer loan	1
Debt collection	1
Credit reporting	1
Credit reporting	1
Debt collection	1
Mortgage	1
Credit reporting	1
Credit reporting	1
Debt collection	1
Debt collection	1
Debt collection	1
Credit reporting	1
Mortgage	1
Debt collection	1
Bank account or service	1
Credit reporting	1
Debt collection	1
Credit reporting	1
Consumer loan	1
Credit reporting	1
Bank account or service	1
Debt collection	1
Bank account or service	1
Mortgage	1
Credit reporting	1
Credit reporting	1
Bank account or service	1
Debt collection	1
Debt collection	1
Debt collection	1
Credit reporting	1
Debt collection	1
Debt collection	1
Debt collection	1
Credit card	1
Credit reporting	1
Credit reporting	1
Credit reporting	1
Credit reporting	1
Credit reporting	1
Debt collection	1
Credit reporting	1
Credit reporting	1
Debt collection	1
Debt collection	1
Credit c

In [1]:
# Read the Consumer_Complaints.csv input file into HDFS
!/usr/local/hadoop/bin/hdfs dfs -copyFromLocal test_strings.txt /user/hadoop

16/05/27 16:59:11 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [62]:
%%writefile Imapper.py
#!/usr/bin/env python
# Create a counter called Mapper Counters that will count the calls to function
sys.stderr.write("reporter:counter:Mapper Counters,Identity Calls,1\n")
import sys
for line in sys.stdin:    
    print("%s" % (line.strip()))

Overwriting Imapper.py


In [65]:
%%writefile Ireducer.py
#!/usr/bin/env python
import sys
# Create a counter called Reducer Counters that will count the calls to function
sys.stderr.write("reporter:counter:Reducer Counters,Identity Calls,1\n")
for line in sys.stdin:    
    print("%s" % (line.strip()))

Overwriting Ireducer.py
